In [29]:
#!/usr/bin/env python
#Andrea Burns, Project train and test
import os, sys, csv
import time
import scipy
import numpy as np
import _pickle as pickle
import getpass

In [30]:
def term_frequency_mat(codebook_lengh, wordids, wordcnts):
    term_freq = []
    for counter, (ids, cnts) in enumerate(zip(wordids, wordcnts)):
        vec = np.array([0] * codebook_lengh)
        for i, cnt in zip(ids, cnts):
            vec[i] = cnt
        term_freq.append(vec)
        
    feature_space = np.vstack(term_freq)
    return feature_space

In [40]:
def load_term_frequency():

    # ****************************************************************************************************
    # load word counts
    # ****************************************************************************************************
    directory = './'

    with open("codebook_data.p", 'rb') as f:
        loaded_data = pickle.load(f)
        
    code_book = loaded_data[0]
    graphlets = loaded_data[1]
    codebook_lengh = len(code_book)

    uuids, wordids, wordcts = [], [], []
    video_uuids, true_labels = [], []
    num_of_vids = 493
    
    for task in range(1, num_of_vids):
        file = open("vid" + str(task) + ".p",'rb')
        (uuid, label, ids, histogram) = pickle.load(file, encoding='latin1')
        wordids.append(ids)
        wordcts.append(histogram)
        video_uuids.append(uuid)
        true_labels.append(label)
    
    online_data = wordids, wordcts, true_labels, video_uuids
    term_freq = term_frequency_mat(codebook_lengh, wordids, wordcts)
    return term_freq, true_labels#online_data, code_book, graphlets

In [41]:
(a,b) = load_term_frequency()

directory:  ./


In [43]:
print(np.unique(b))

['microwaving_food' 'openning_double_doors' 'printing_interface'
 'printing_take_printout' 'take_from_fridge' 'take_paper_towel'
 'take_tea_bag' 'throw_trash' 'use_kettle' 'use_water_cooler' 'washing_up']


In [48]:
class_labels = []
for i in range(0,len(b)):
    if b[i] == 'microwaving_food':
        class_labels.append(0)
    if b[i] == 'openning_double_doors':
        class_labels.append(1)
    if b[i] == 'printing_interface':
        class_labels.append(2)
    if b[i] == 'printing_take_printout':
        class_labels.append(3)
    if b[i] == 'take_from_fridge':
        class_labels.append(4)
    if b[i] == 'take_paper_towel':
        class_labels.append(5)
    if b[i] == 'take_tea_bag':
        class_labels.append(6)
    if b[i] == 'throw_trash':
        class_labels.append(7)
    if b[i] == 'use_kettle':
        class_labels.append(8)
    if b[i] == 'use_water_cooler':
        class_labels.append(9)
    if b[i] == 'washing_up':
        class_labels.append(10)

In [32]:
def high_instance_code_words(term_frequency, code_book, graphlets, low_instance):
    """This essentially takes the feature space created over all events, and removes any
    feature that is not witnessed in a minimum number of observations (low_instance param).
    """
    ## Number of rows with non zero element :
    keep_rows = np.where((term_frequency != 0).sum(axis=0) > low_instance)[0]   # removes code words if they dont appear in a minimum number of videos
    # keep_rows = np.where(term_frequency.sum(axis=0) > low_instance)[0]        # removes code words if they dont appear a minimum number of times across all videos

    ## Sum of the whole column: term_frequency.sum(axis=0) > low_instance
    remove_inds = np.where((term_frequency != 0).sum(axis=0) <= low_instance)[0]

    print("orig feature space: %s. remove: %s. new space: %s." % (len(term_frequency.sum(axis=0)), len(remove_inds), len(keep_rows)))

    #keep only the columns of the feature space which have more than low_instance number of occurances.
    selected_features = term_frequency.T[keep_rows]
    new_term_frequency = selected_features.T
    print("new feature space shape: ", new_term_frequency.shape)

    # # Code Book (1d np array of hash values)
    new_code_book = code_book[keep_rows]
    print("  new code book len: ", len(new_code_book))

    # # Graphlets book (1d np array of igraphs)
    new_graphlets = graphlets[keep_rows]
    print("  new graphlet book len: ", len(new_graphlets))

    print("removed low (%s) instance graphlets" % low_instance)
    print("shape = ", new_term_frequency.shape)

    return new_term_frequency, new_code_book, new_graphlets

In [34]:
(a,b,c,d) = load_term_frequency()

directory:  ./


In [36]:
(e,f,g) = high_instance_code_words(a,c,d,1)

orig feature space: 22829. remove: 12640. new space: 10189.
new feature space shape:  (492, 10189)
  new code book len:  10189
  new graphlet book len:  10189
removed low (1) instance graphlets
shape =  (492, 10189)


In [50]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(e, class_labels, test_size=0.2, random_state=42)

In [53]:
from sklearn import svm
classifier = svm.SVC(kernel='linear', C=.1).fit(X_train, y_train)
predicted = classifier.predict(X_test)

In [54]:
accuracy = 0
for i in range(0,len(predicted)):
    if predicted[i] == y_test[i]:
        accuracy += 1
accuracy = accuracy/len(predicted)
print(accuracy)

0.797979797979798
